In [244]:
import pyspark
from operator import add
from pyspark import SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col,udf
from pyspark.sql import SQLContext,Row
from operator import add
import numpy as np
import string
from pyspark.ml.feature import HashingTF, IDF, Tokenizer,NGram,CountVectorizer
from pyspark.ml.feature import StringIndexer
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.linalg import VectorUDT as VectorUDTML
from pyspark.sql.functions import udf

as_ml = udf(lambda v: v.asML() if v is not None else None, VectorUDTML())

#sc = pyspark.SparkContext('local[*]',appName="DocClassification")
#sqlc = SQLContext(sc)



In [212]:
data = sc.parallelize(["This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something."])\
            .zipWithIndex()\
            .toDF(['doc','did'])

data.show()
labels = sc.parallelize([(1,0),(2,1),(3,1),(4,0),(5,0),(6,1),(7,0),(8,1),(9,1),(10,0),(11,0),(12,1)])\
            .map(lambda x: (x[0]-1,x[1])).toDF(['did','lab'])
labels.show()

+--------------------+---+
|                 doc|did|
+--------------------+---+
|This is something...|  0|
|To test something...|  1|
|I need to do some...|  2|
|This is something...|  3|
|To test something...|  4|
|I need to do some...|  5|
|This is something...|  6|
|To test something...|  7|
|I need to do some...|  8|
|This is something...|  9|
|To test something...| 10|
|I need to do some...| 11|
+--------------------+---+

+---+---+
|did|lab|
+---+---+
|  0|  0|
|  1|  1|
|  2|  1|
|  3|  0|
|  4|  0|
|  5|  1|
|  6|  0|
|  7|  1|
|  8|  1|
|  9|  0|
| 10|  0|
| 11|  1|
+---+---+



In [213]:
toker = Tokenizer(inputCol = "doc",outputCol = "words")
data = toker.transform(data)
grammer = NGram(n=2,inputCol="words",outputCol="grams")
data = grammer.transform(data).drop('doc')
data.show()

+---+--------------------+--------------------+
|did|               words|               grams|
+---+--------------------+--------------------+
|  0|[this, is, someth...|[this is, is some...|
|  1|[to, test, someth...|[to test, test so...|
|  2|[i, need, to, do,...|[i need, need to,...|
|  3|[this, is, someth...|[this is, is some...|
|  4|[to, test, someth...|[to test, test so...|
|  5|[i, need, to, do,...|[i need, need to,...|
|  6|[this, is, someth...|[this is, is some...|
|  7|[to, test, someth...|[to test, test so...|
|  8|[i, need, to, do,...|[i need, need to,...|
|  9|[this, is, someth...|[this is, is some...|
| 10|[to, test, someth...|[to test, test so...|
| 11|[i, need, to, do,...|[i need, need to,...|
+---+--------------------+--------------------+



In [214]:
data = data.rdd.map(lambda x: Row(x['did'],x['words']+x['grams'])).toDF(['did','features'])
data.show()


+---+--------------------+
|did|            features|
+---+--------------------+
|  0|[this, is, someth...|
|  1|[to, test, someth...|
|  2|[i, need, to, do,...|
|  3|[this, is, someth...|
|  4|[to, test, someth...|
|  5|[i, need, to, do,...|
|  6|[this, is, someth...|
|  7|[to, test, someth...|
|  8|[i, need, to, do,...|
|  9|[this, is, someth...|
| 10|[to, test, someth...|
| 11|[i, need, to, do,...|
+---+--------------------+



In [215]:
cv = CountVectorizer(inputCol="features", outputCol="featureVecs")
data = cv.fit(data).transform(data)

You can very easily add the labels onto your DF if you already have them joined in...

In [216]:
data.show()

+---+--------------------+--------------------+
|did|            features|         featureVecs|
+---+--------------------+--------------------+
|  0|[this, is, someth...|(37,[0,1,2,3,4,5,...|
|  1|[to, test, someth...|(37,[0,1,2,3,4,5,...|
|  2|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|
|  3|[this, is, someth...|(37,[0,1,2,3,4,5,...|
|  4|[to, test, someth...|(37,[0,1,2,3,4,5,...|
|  5|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|
|  6|[this, is, someth...|(37,[0,1,2,3,4,5,...|
|  7|[to, test, someth...|(37,[0,1,2,3,4,5,...|
|  8|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|
|  9|[this, is, someth...|(37,[0,1,2,3,4,5,...|
| 10|[to, test, someth...|(37,[0,1,2,3,4,5,...|
| 11|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|
+---+--------------------+--------------------+



In [217]:
data = data.join(labels,['did'])
data.show()

+---+--------------------+--------------------+---+
|did|            features|         featureVecs|lab|
+---+--------------------+--------------------+---+
|  0|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|
|  7|[to, test, someth...|(37,[0,1,2,3,4,5,...|  1|
|  6|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|
|  9|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|
|  5|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|
|  1|[to, test, someth...|(37,[0,1,2,3,4,5,...|  1|
| 10|[to, test, someth...|(37,[0,1,2,3,4,5,...|  0|
|  3|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|
|  8|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|
| 11|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|
|  2|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|
|  4|[to, test, someth...|(37,[0,1,2,3,4,5,...|  0|
+---+--------------------+--------------------+---+



In [218]:
ddata = data.rdd
ddata= ddata.map(lambda x: tuple(x))
ddata.collect()

[(0,
  ['this',
   'is',
   'something',
   'i',
   'really',
   'want',
   'to',
   'test.',
   'this is',
   'is something',
   'something i',
   'i really',
   'really want',
   'want to',
   'to test.'],
  SparseVector(37, {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 9: 1.0, 11: 1.0, 13: 1.0, 14: 1.0, 27: 1.0, 30: 1.0, 31: 1.0, 34: 1.0}),
  0),
 (7,
  ['to',
   'test',
   'something',
   'is',
   'something',
   'i',
   'really',
   'want',
   'to',
   'do.',
   'to test',
   'test something',
   'something is',
   'is something',
   'something i',
   'i really',
   'really want',
   'want to',
   'to do.'],
  SparseVector(37, {0: 2.0, 1: 2.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 13: 1.0, 14: 1.0, 20: 1.0, 22: 1.0, 29: 1.0, 32: 1.0}),
  1),
 (6,
  ['this',
   'is',
   'something',
   'i',
   'really',
   'want',
   'to',
   'test.',
   'this is',
   'is something',
   'something i',
   'i really',
   'really want',
   'want to',
   '

In [219]:
indexer = StringIndexer(inputCol="lab", outputCol="indexedLabel")
data = indexer.fit(data).transform(data)
data.show()

+---+--------------------+--------------------+---+------------+
|did|            features|         featureVecs|lab|indexedLabel|
+---+--------------------+--------------------+---+------------+
|  0|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|         0.0|
|  7|[to, test, someth...|(37,[0,1,2,3,4,5,...|  1|         1.0|
|  6|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|         0.0|
|  9|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|         0.0|
|  5|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|         1.0|
|  1|[to, test, someth...|(37,[0,1,2,3,4,5,...|  1|         1.0|
| 10|[to, test, someth...|(37,[0,1,2,3,4,5,...|  0|         0.0|
|  3|[this, is, someth...|(37,[0,1,2,3,4,5,...|  0|         0.0|
|  8|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|         1.0|
| 11|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|         1.0|
|  2|[i, need, to, do,...|(37,[0,1,2,3,4,5,...|  1|         1.0|
|  4|[to, test, someth...|(37,[0,1,2,3,4,5,...|  0|         0.0|
+---+--------------------

In [222]:
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="featureVecs", numTrees=10)
model = rf.fit(data)


In [245]:
data2 = data.rdd\
            .map(lambda x: tuple(x))\
            .map(lambda x: (x[3],x[2]))\
            .map(lambda x: LabeledPoint(x[0],MLLibVectors.fromML(x[1])))\
            .toDF()
data2 = data2.withColumn("features", as_ml("features"))

In [246]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(data2)